In [1]:
import pandas as pd

# Load Events Data from Retrosheet

Raw retrosheet data contains event logs, representing raw information about the events in a baseball game.  In compiling the dataset into a useable form, the software Chadwick (found here:http://chadwick.sourceforge.net/doc/index.html) was used to convert the raw logs into CSV.  Luckily for us, Chadwick computes some pretty important quantities that we can pull really easily into a table.

For computing the Run Expectancy Matrix, we only need a few columns.  The relevant columns are:
+ OUTS_CT - number of outs
+ START_BASES_CD - a code representing the state of the runners, eg. runner on 2nd
+ EVENT_RUNS_CT - number of runs scored on this event
+ FATE_RUNS_CT - number of runs scored AFTER this event

In [2]:
base_runner_codes = {
    0: "OOO", # No one on
    1: "XOO", # runner on 1st
    2: "OXO", # runner on 2nd
    3: "XXO", # runners on 1st & 2nd
    4: "OOX", # runner on 3d
    5: "XOX", # runners on 1st & 3rd
    6: "OXX", # runners on 2nd & 3rd
    7: "XXX"  # bases loaded
}

In [3]:
cols = ['OUTS_CT', 'START_BASES_CD', 'EVENT_RUNS_CT', 'FATE_RUNS_CT']
events = pd.read_csv('../data/mlb_pxp/csv/events-2016.csv.gz', usecols=cols)
events['START_BASES_CD'].replace(base_runner_codes, inplace=True)

# Compute Runs in Remainder of Inning (ROI)

The Run Expectancy for an Out-Baserunner state (say, 1 out, runners on 1st and 2nd) is computed as the average number of runs scored during and following the plate appearances in that state.  To compute the ROI, we just need to add up FATE_RUNS_CT and EVENT_RUNS_CT.

In [4]:
events['RUNS_ROI_CT'] = events['FATE_RUNS_CT'] + events['EVENT_RUNS_CT']

# Run Expectancy Matrix

Finally, we need to group by the out count and the state of the runners and take the average.

In [5]:
events.groupby(['OUTS_CT', 'START_BASES_CD'])['RUNS_ROI_CT'].\
    mean().\
    unstack(level=0)

OUTS_CT,0,1,2
START_BASES_CD,,,
OOO,0.502513,0.270856,0.107196
OOX,1.343629,0.939116,0.374473
OXO,1.135514,0.677241,0.316581
OXX,1.931343,1.367451,0.554517
XOO,0.864710,0.517304,0.222876
XOX,1.719588,1.198185,0.482832
XXO,1.449766,0.926776,0.418962
XXX,2.103841,1.541667,0.705135
